<a href="https://colab.research.google.com/github/open-mmlab/mmpose/blob/dev-1.x/demo/MMPose_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Check Pytorch installation
import torch, torchvision

print('torch version:', torch.__version__, torch.cuda.is_available())
print('torchvision version:', torchvision.__version__)

# Check MMPose installation
import mmpose

print('mmpose version:', mmpose.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version

print('cuda version:', get_compiling_cuda_version())
print('compiler information:', get_compiler_version())

torch version: 2.0.1+cu118 True
torchvision version: 0.15.2+cu118
mmpose version: 1.0.0
cuda version: 11.8
compiler information: MSVC 192829924


## Inference with an MMPose model

MMPose provides high-level APIs for model inference and training.

In [5]:
import os
import json
from PIL import Image

In [6]:
import mmcv
from mmcv import imread
import mmengine
from mmengine.registry import init_default_scope
import numpy as np

from mmpose.apis import inference_topdown
from mmpose.apis import init_model as init_pose_estimator
from mmpose.evaluation.functional import nms
from mmpose.registry import VISUALIZERS
from mmpose.structures import merge_data_samples

try:
    from mmdet.apis import inference_detector, init_detector
    has_mmdet = True
except (ImportError, ModuleNotFoundError):
    has_mmdet = False

local_runtime = False

try:
    from google.colab.patches import cv2_imshow  # for image visualization in colab
except:
    local_runtime = True

# config파일 경로 에러 문구 확인해서 잘 바꿔써주세요.
pose_config = 'C:/Users/haeinlee/mmpose/configs/body_2d_keypoint/topdown_heatmap/coco/td-hm_hrnet-w32_8xb64-210e_coco-256x192.py'
pose_checkpoint = 'https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w32_coco_256x192-c78dce93_20200708.pth'
det_config = 'C:/Users/haeinlee/mmpose/demo/mmdetection_cfg/faster_rcnn_r50_fpn_coco.py'
det_checkpoint = 'https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth'

device = 'cuda:0'
cfg_options = dict(model=dict(test_cfg=dict(output_heatmaps=True)))


# build detector
detector = init_detector(
    det_config,
    det_checkpoint,
    device=device
)


# build pose estimator
pose_estimator = init_pose_estimator(
    pose_config,
    pose_checkpoint,
    device=device,
    cfg_options=cfg_options
)

# init visualizer
pose_estimator.cfg.visualizer.radius = 3
pose_estimator.cfg.visualizer.line_width = 1
visualizer = VISUALIZERS.build(pose_estimator.cfg.visualizer)
# the dataset_meta is loaded from the checkpoint and
# then pass to the model in init_pose_estimator
visualizer.set_dataset_meta(pose_estimator.dataset_meta)



#####----------------------------------------------------------------------------------------------------------------------------------------------###




Loads checkpoint by http backend from path: https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth
Loads checkpoint by http backend from path: https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w32_coco_256x192-c78dce93_20200708.pth
06/19 05:33:50 - mmengine - WARNING - `Visualizer` backend is not initialized because save_dir is None.


c:\users\haeinlee\mmpose\demo\mmpose\mmpose\datasets\datasets\utils.py:102: UserWarning: The metainfo config file "configs/_base_/datasets/coco.py" does not exist. A matched config file "c:\users\haeinlee\mmpose\demo\mmpose\mmpose\.mim\configs\_base_\datasets\coco.py" will be used instead.
  warnings.warn(


#모델가져오기

In [7]:
import tensorflow as tf
model = tf.keras.models.load_model('./model9')

# Writing Data
관절 위치 추정하여 화질과 함께 data작성

In [3]:
img_root = './test1/'
#test 1 : pro05, test 2: pub01 test 3: pro08, test 4 : pub03
data_flag = 0
directory_data=[]

for (path, dir, files) in os.walk(img_root):
    for f in files:
        img = img_root+f
        image1 = Image.open(img)
        
        init_default_scope(detector.cfg.get('default_scope', 'mmdet'))
        detect_result = inference_detector(detector, img)
        pred_instance = detect_result.pred_instances.cpu().numpy()
        bboxes = np.concatenate(
            (pred_instance.bboxes, pred_instance.scores[:, None]), axis=1)
        bboxes = bboxes[np.logical_and(pred_instance.labels == 0,
                                        pred_instance.scores > 0.3)]
        bboxes = bboxes[nms(bboxes, 0.3)][:, :4]
        inference_result = inference_topdown(pose_estimator, img, bboxes)
        
        tmp = np.array(sum(inference_result[0].pred_instances.keypoints[0][5:].tolist(),[]))
        tmp[0::2] = tmp[0::2]/image1.size[0]
        tmp[1::2] = tmp[1::2]/image1.size[1]
        
        directory_data.append(tmp)
              
    if data_flag==0:
        x=np.array([directory_data])
        data_flag=1
        
    else:
        x= np.concatenate((x, np.array([directory_data])),axis=0)
        
        
print(model.predict(x))

NameError: name 'os' is not defined

In [5]:
import tensorflow as tf
model = tf.keras.models.load_model('./simple_LSTM')

In [8]:
img_root = 'C:/Users/haeinlee/Downloads/test_data-20230618T092038Z-001/test_data/public/'
#test 1 : pro05, test 2: pub01 test 3: pro08, test 4 : pub03


for (root_path, dir, files) in os.walk(img_root):
    for d in dir:
        for (path, dir, files) in os.walk(img_root+d):
            data_flag = 0
            directory_data=[]
            for f in files:
                img = path+'/'+f
                image1 = Image.open(img)

                init_default_scope(detector.cfg.get('default_scope', 'mmdet'))
                detect_result = inference_detector(detector, img)
                pred_instance = detect_result.pred_instances.cpu().numpy()
                bboxes = np.concatenate(
                    (pred_instance.bboxes, pred_instance.scores[:, None]), axis=1)
                bboxes = bboxes[np.logical_and(pred_instance.labels == 0,
                                                pred_instance.scores > 0.3)]
                bboxes = bboxes[nms(bboxes, 0.3)][:, :4]
                inference_result = inference_topdown(pose_estimator, img, bboxes)

                tmp = np.array(sum(inference_result[0].pred_instances.keypoints[0][5:].tolist(),[]))
                tmp[0::2] = tmp[0::2]/image1.size[0]
                tmp[1::2] = tmp[1::2]/image1.size[1]

                directory_data.append(tmp)

            if data_flag==0:
                x=np.array([directory_data])
                data_flag=1

            else:
                x= np.concatenate((x, np.array([directory_data])),axis=0)


        print(model.predict(x))

06/19 05:33:57 - mmengine - WARNING - The current default scope "mmpose" is not "mmdet", `init_default_scope` will force set the currentdefault scope to "mmdet".
06/19 05:34:10 - mmengine - WARNING - The current default scope "mmdet" is not "mmpose", `init_default_scope` will force set the currentdefault scope to "mmpose".
1/1 [==============================] - 2s 2s/step
[[0.0951936]]
1/1 [==============================] - 0s 341ms/step
[[0.07814419]]
1/1 [==============================] - 0s 20ms/step
[[0.03763203]]
1/1 [==============================] - 0s 19ms/step
[[0.01091004]]
1/1 [==============================] - 0s 21ms/step
[[0.15036374]]
1/1 [==============================] - 0s 18ms/step
[[0.00835619]]
1/1 [==============================] - 0s 40ms/step
[[0.0881592]]
1/1 [==============================] - 0s 21ms/step
[[0.06345508]]
1/1 [==============================] - 0s 21ms/step
[[0.0304906]]
1/1 [==============================] - 0s 18ms/step
[[0.02432534]]


In [9]:
img_root = 'C:/Users/haeinlee/Downloads/test_data-20230618T092038Z-001/test_data/pro/'
#test 1 : pro05, test 2: pub01 test 3: pro08, test 4 : pub03


for (root_path, dir, files) in os.walk(img_root):
    for d in dir:
        for (path, dir, files) in os.walk(img_root+d):
            data_flag = 0
            directory_data=[]
            for f in files:
                img = path+'/'+f
                image1 = Image.open(img)

                init_default_scope(detector.cfg.get('default_scope', 'mmdet'))
                detect_result = inference_detector(detector, img)
                pred_instance = detect_result.pred_instances.cpu().numpy()
                bboxes = np.concatenate(
                    (pred_instance.bboxes, pred_instance.scores[:, None]), axis=1)
                bboxes = bboxes[np.logical_and(pred_instance.labels == 0,
                                                pred_instance.scores > 0.3)]
                bboxes = bboxes[nms(bboxes, 0.3)][:, :4]
                inference_result = inference_topdown(pose_estimator, img, bboxes)

                tmp = np.array(sum(inference_result[0].pred_instances.keypoints[0][5:].tolist(),[]))
                tmp[0::2] = tmp[0::2]/image1.size[0]
                tmp[1::2] = tmp[1::2]/image1.size[1]

                directory_data.append(tmp)

            if data_flag==0:
                x=np.array([directory_data])
                data_flag=1

            else:
                x= np.concatenate((x, np.array([directory_data])),axis=0)


        print(model.predict(x))

1/1 [==============================] - 0s 32ms/step
[[0.32485116]]
1/1 [==============================] - 0s 41ms/step
[[0.01582544]]
1/1 [==============================] - 0s 27ms/step
[[0.00501433]]
1/1 [==============================] - 0s 18ms/step
[[0.5536788]]
1/1 [==============================] - 0s 19ms/step
[[0.02573157]]
1/1 [==============================] - 0s 22ms/step
[[0.43740433]]
1/1 [==============================] - 0s 22ms/step
[[0.00831002]]
1/1 [==============================] - 0s 25ms/step
[[0.21196523]]
